In [1]:
import math
import pandas as pd
import os

import glob
from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq
import time


import psycopg2 as pg2
from sqlalchemy import create_engine

from datetime import datetime
from datetime import timedelta

import os
import time

import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


In [2]:
# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


### GCS 데이터 등록

In [3]:

# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'

# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


### csv, parquet

- [parquet 공식문서](https://arrow.apache.org/docs/python/parquet.html)
- [Python parquet 타입으로 읽기, 쓰기](https://owening2.tistory.com/19)
- [parquet? / csv, parquet 파일 불러오기 테스트](https://velog.io/@freetix/parquet-csv-parquet-%ED%8C%8C%EC%9D%BC-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0-%ED%85%8C%EC%8A%A4%ED%8A%B8
)


In [4]:
os.getcwd()

'/home/shjj08choi4/finance_mlops'

In [5]:
kor_stock_ohlcv = pd.read_csv('data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.csv')

/tmp/ipykernel_883218/2625704097.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  kor_stock_ohlcv = pd.read_csv('data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.csv')


In [24]:
kor_ticker_list = pd.read_csv("data_crawler/kor_ticker_list/kor_ticker_list.csv")

kor_stock_fundamental = pd.read_csv("data_crawler/kor_stock_fundamental/kor_stock_fundamental.csv")


/tmp/ipykernel_883218/2503545936.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  kor_stock_fundamental = pd.read_csv("data_crawler/kor_stock_fundamental/kor_stock_fundamental.csv")


In [13]:
kor_stock_ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3366395 entries, 0 to 3366394
Data columns (total 8 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   date                     object 
 1   open                     int64  
 2   high                     int64  
 3   low                      int64  
 4   close                    int64  
 5   volume                   int64  
 6   price_change_percentage  float64
 7   ticker                   object 
dtypes: float64(1), int64(5), object(2)
memory usage: 205.5+ MB


In [13]:
kor_stock_ohlcv.head()

,date,open,high,low,close,volume,price_change_percentage,ticker
0,2018-01-02,6980,6990,6850,6970,60294,0.000000,095570
1,2018-01-03,6970,7000,6910,6920,45848,-0.717360,095570
2,2018-01-04,6990,7100,6920,7070,53176,2.167630,095570
3,2018-01-05,7080,7300,7040,7250,58573,2.545969,095570
4,2018-01-08,7330,7330,7100,7130,47104,-1.655172,095570


In [25]:
kor_stock_ohlcv['ticker'] = kor_stock_ohlcv['ticker'].astype('str')

kor_ticker_list['ticker'] = kor_ticker_list['ticker'].astype('str')
kor_stock_fundamental['ticker'] = kor_stock_fundamental['ticker'].astype('str')



#### parquet로 저장하기

In [15]:

table_from_pandas = pa.Table.from_pandas(kor_stock_ohlcv,preserve_index = False)
pq.write_table(table_from_pandas, 'data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet')

/home/shjj08choi4/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [26]:

table_from_pandas = pa.Table.from_pandas(kor_ticker_list,preserve_index = False)
pq.write_table(table_from_pandas, 'data_crawler/kor_ticker_list/kor_ticker_list.parquet')

/home/shjj08choi4/anaconda3/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [27]:

table_from_pandas = pa.Table.from_pandas(kor_stock_fundamental,preserve_index = False)
pq.write_table(table_from_pandas, 'data_crawler/kor_stock_fundamental/kor_stock_fundamental.parquet')

#### parquet 읽어오기

In [7]:
t = pq.read_table('data_crawler/kor_stock_ohlcv/kor_stock_ohlcv.parquet')
kor_stock_ohlcv = t.to_pandas()

In [8]:
t = pq.read_table('data_crawler/kor_stock_fundamental/kor_stock_fundamental.parquet')
kor_stock_fundamental = t.to_pandas()

In [9]:
t = pq.read_table('data_crawler/kor_ticker_list/kor_ticker_list.parquet')
kor_ticker_list = t.to_pandas()

,ticker,corp_name,market
0,095570,AJ네트웍스,KOSPI
1,006840,AK홀딩스,KOSPI
2,027410,BGF,KOSPI
3,282330,BGF리테일,KOSPI
4,138930,BNK금융지주,KOSPI
...,...,...,...
947,079980,휴비스,KOSPI
948,005010,휴스틸,KOSPI
949,000540,흥국화재,KOSPI
950,000545,흥국화재우,KOSPI


In [15]:
kor_ticker_list_kospi = kor_ticker_list[kor_ticker_list['market'] == 'KOSPI'].reset_index(drop = True)
kor_ticker_list_kosdaq = kor_ticker_list[kor_ticker_list['market'] == 'KOSDAQ'].reset_index(drop = True)

In [28]:
kor_stock_fundamental_info = pd.merge(kor_stock_fundamental, kor_ticker_list, 
        on = 'ticker', 
        how = 'left')

In [30]:
kor_stock_fundamental_info

,date,bps,per,pbr,eps,div,dps,ticker,corp_name,market
0,2018-01-02,5980,24.20,1.17,288,0.86,60,095570,AJ네트웍스,KOSPI
1,2018-01-03,5980,24.03,1.16,288,0.87,60,095570,AJ네트웍스,KOSPI
2,2018-01-04,5980,24.55,1.18,288,0.85,60,095570,AJ네트웍스,KOSPI
3,2018-01-05,5980,25.17,1.21,288,0.83,60,095570,AJ네트웍스,KOSPI
4,2018-01-08,5980,24.76,1.19,288,0.84,60,095570,AJ네트웍스,KOSPI
...,...,...,...,...,...,...,...,...,...,...
3278193,2023-11-30,5656,0.00,1.12,0,0.00,0,238490,힘스,KOSDAQ
3278194,2023-12-01,5656,0.00,1.13,0,0.00,0,238490,힘스,KOSDAQ
3278195,2023-12-04,5656,0.00,1.14,0,0.00,0,238490,힘스,KOSDAQ
3278196,2023-12-05,5656,0.00,1.12,0,0.00,0,238490,힘스,KOSDAQ


In [31]:

kor_stock_fundamental_info_kospi = kor_stock_fundamental_info[kor_stock_fundamental_info['market'] == 'KOSPI'].reset_index(drop = True)
kor_stock_fundamental_info_kosdaq = kor_stock_fundamental_info[kor_stock_fundamental_info['market'] == 'KOSDAQ'].reset_index(drop = True)

In [ ]:
kor_stock_fundamental_info_kospi

In [33]:


table_from_pandas = pa.Table.from_pandas(kor_stock_fundamental_info_kosdaq,preserve_index = False)
pq.write_table(table_from_pandas, 'data_crawler/kor_stock_fundamental/kor_stock_fundamental_info_kosdaq.parquet')

In [ ]:
kor_stock_ohlcv_info_kospi
kor_stock_ohlcv_info_kosdaq

### KOSPI KOSDAQ 분류

In [35]:
# Google Storage 적재
source_file_name = f'data_crawler/kor_stock_fundamental/kor_stock_fundamental_info_kospi.parquet'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/kor_stock_fundamental/kor_stock_fundamental_info_kospi.parquet'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)